***Transfer Learning Method***

In [1]:
!pip install keras-vggface
!pip install scikit-image
!pip install pydot

In [2]:
!pip install keras_applications
!pip install keras_preprocessing

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline

import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras_vggface.vggface import VGGFace
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D
import skimage
from skimage.transform import rescale, resize

import pydot

import zipfile

In [4]:
print(tf.__version__)
print(keras.__version__)

2.4.1
2.4.3


Define Parameters

In [5]:
EPOCHS = 100
BS = 128
DROPOUT_RATE = 0.5
FROZEN_LAYER_NUM = 201

ADAM_LEARNING_RATE = 0.001
SGD_LEARNING_RATE = 0.01
SGD_DECAY = 0.0001
#stochastic gradient descent
Resize_pixelsize = 197

Build Model

In [6]:
vgg_notop = VGGFace(model='senet50', include_top=False, input_shape=(Resize_pixelsize, Resize_pixelsize, 3), pooling='avg')
last_layer = vgg_notop.get_layer('avg_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dropout(DROPOUT_RATE)(x) ## dropout prevent overfitting in neural network
x = Dense(4096, activation='relu', name='fc6')(x)
x = Dropout(DROPOUT_RATE)(x)
x = Dense(1024, activation='relu', name='fc7')(x)
x = Dropout(DROPOUT_RATE)(x)

In [7]:
batch_norm_indices = [2, 6, 9, 12, 21, 25, 28, 31, 42, 45, 48, 59, 62, 65, 74, 78, 81, 84, 95, 98, 101, 112, 115, 118, 129, 132, 135, 144, 148, 151, 154, 165, 168, 171, 182, 185, 188, 199, 202, 205, 216, 219, 222, 233, 236, 239, 248, 252, 255, 258, 269, 272, 275]    
for i in range(FROZEN_LAYER_NUM):
    if i not in batch_norm_indices:
        vgg_notop.layers[i].trainable = False
    

In [8]:
out = Dense(7, activation='softmax', name='classifier')(x)

model = Model(vgg_notop.input, out)

optim = keras.optimizers.Adam(lr=ADAM_LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
#optim = keras.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
sgd = keras.optimizers.SGD(lr=SGD_LEARNING_RATE, momentum=0.9, decay=SGD_DECAY, nesterov=True)
rlrop = keras.callbacks.ReduceLROnPlateau(monitor='val_acc',mode='max',factor=0.5, patience=10, min_lr=0.00001, verbose=1) 
#ReduceLROnPlateau回调函数， 训练过程种优化学习率

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

Decompressing Files

In [9]:
#import zipfile

In [10]:
def unzip_file(zip_src, dst_dir):
    r = zipfile.is_zipfile(zip_src)
    if r:     
        fz = zipfile.ZipFile(zip_src, 'r')
        for file in fz.namelist():
            fz.extract(file, dst_dir)       
    else:
        print('This is not zip')
        
        
        
## zip_src refers to the complete path of the zip file; and dst_dir refers to the directory in where you want to put the unzipped files. 

In [27]:
unzip_file(r'C:\Users\59606\Desktop\train.zip',r'C:\Users\59606\Desktop\FER_Data\train')

In [30]:
unzip_file(r'C:\Users\59606\Desktop\test-public.zip',r'C:\Users\59606\Desktop\FER_Data\dev')

In [31]:
unzip_file(r'C:\Users\59606\Desktop\test-private.zip',r'C:\Users\59606\Desktop\FER_Data\test')

Generate the imagedata

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_datagen(dataset, aug=False):
    if aug:
        datagen = ImageDataGenerator(
                            rescale=1./255,
                            featurewise_center=False,
                            featurewise_std_normalization=False,
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True)
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow_from_directory(
            dataset,
            target_size=(197, 197),
            color_mode='rgb',
            shuffle = True,
            class_mode='categorical',
            batch_size=BS)

In [12]:
train_generator  = get_datagen(r'C:\Users\59606\Desktop\FER_Data\train', True)

Found 28709 images belonging to 7 classes.


In [13]:
dev_generator    = get_datagen(r'C:\Users\59606\Desktop\FER_Data\dev')
test_generator  = get_datagen(r'C:\Users\59606\Desktop\FER_Data\test')

Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


Training the Model

In [ ]:
history = model.fit_generator(
    generator = train_generator,
    validation_data=dev_generator, 
    #steps_per_epoch=28709// BS,
    #validation_steps=3509 // BS,
    shuffle=True,
    epochs=2,
    callbacks=[rlrop],
    use_multiprocessing=True,
) 

C:\Anaconda\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2


Exception in thread Thread-73:
Traceback (most recent call last):
  File "C:\Anaconda\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Anaconda\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Anaconda\lib\site-packages\tensorflow\python\keras\utils\data_utils.py", line 748, in _run
    with closing(self.executor_fn(_SHARED_SEQUENCES)) as executor:
  File "C:\Anaconda\lib\site-packages\tensorflow\python\keras\utils\data_utils.py", line 725, in pool_fn
    pool = get_pool_class(True)(
  File "C:\Anaconda\lib\multiprocessing\context.py", line 119, in Pool
    return Pool(processes, initializer, initargs, maxtasksperchild,
  File "C:\Anaconda\lib\multiprocessing\pool.py", line 212, in __init__
    self._repopulate_pool()
  File "C:\Anaconda\lib\multiprocessing\pool.py", line 303, in _repopulate_pool
    return self._repopulate_pool_static(self._ctx, self.Process,
  File "C:\Anaconda\lib\multiprocessing\pool.py", line 32

Evaluate the Result

In [ ]:
print('\n# Evaluate on dev data')
results_dev = model.evaluate_generator(dev_generator, 3509 // BS)
print('dev loss, dev acc:', results_dev)

In [ ]:
print('\n# Evaluate on test data')
results_test = model.evaluate_generator(test_generator, 3509 // BS)
print('test loss, test acc:', results_test)

Visualization

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'dev'], loc='upper left')
plt.show()

Save Model

In [ ]:
epoch_str = '-EPOCHS_' + str(EPOCHS)
test_acc = 'test_acc_%.3f' % results_test[1]
model.save('C:\Users\59606\Desktop\FER_Data\model\' + 'SENET50' + epoch_str + test_acc + '.h5')